In [1]:
from sklearn import svm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import csv
from sklearn.preprocessing import OneHotEncoder
import cv2
import os
import json
import math
import time
import torch

from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
from val import normalize, pad_width

NORMAL_LENGTH = 100
FRAME_WIDTH = 1920
KEYPOINTS = [0, 1, 2, 3, 4, 5, 6, 7]
BONES_DRAW = [[0, 1], [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]]
BONES = [[0, 1], [2, 3], [3, 4], [5, 6], [6, 7]]
ANGLES = [[[1, 0], 90], [[3, 2], 180], [[4, 3], 90], [[6, 5], 0], [[7, 6], 90]]
CUT_PHOTO = 30

KEYPOINTS_4dots = [0, 2, 5, 14, 15, 16, 17]
BONES_4dots = [[0, 1]]
ANGLES_4dots = [[[1, 0], 90], [[1, 2], 90], [[1, 5], 90]]
json_name = '00021'
video_name = './訓練用影片/' + json_name + '.MTS'
print(video_name)
kernel = 'linear'
C = 1
gamma = 'auto'

./訓練用影片/00021.MTS


In [2]:
# file_name = './訓練用影片/after_transform_1to15.csv'
# X = []
# y = []

# #讀取訓練資料內容
# with open(file_name, newline='') as csvfile:
#     rows = csv.reader(csvfile)
#     for row in rows:
#         temp = []
#         for i in range(len(row)-2):
#             temp.append(float(row[i]))
#         #將資料'左中右'轉成onehotencoding
#         if row[-2] == 'left':
#             temp.append(1)
#             temp.append(0)
#             temp.append(0)
#         elif row[-2] == 'right':
#             temp.append(0)
#             temp.append(0)
#             temp.append(1)
#         elif row[-2] == 'middle':
#             temp.append(0)
#             temp.append(1)
#             temp.append(0)
#         #把bowing、looking、others都歸類為others
#         if row[-1] == 'bowing' or row[-1] == 'looking' or row[-1] == 'others': y.append('others')
#         else: y.append(row[-1])
#         X.append(temp)
# print(X[0])
# X = np.array(X)
# y = np.array(y)
# print(X.shape)
# print(y.shape)
# print(X[0])
# print(y[0])

In [3]:
file_name = './訓練用影片/after_transform_1to15.csv'
X = []
y = []

#讀取訓練資料內容
with open(file_name, newline='') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        
        temp = []
        for i in range(len(row)-2):
            temp.append(float(row[i]))
        #將資料'左中右'轉成onehotencoding
        if row[-2] == 'left':
            temp.append(1)
            temp.append(0)
            temp.append(0)
        elif row[-2] == 'right':
            temp.append(0)
            temp.append(0)
            temp.append(1)
        elif row[-2] == 'middle':
            temp.append(0)
            temp.append(1)
            temp.append(0)
        #把bowing、looking、others都歸類為others
        if row[-1] == 'bowing' or row[-1] == 'looking' or row[-1] == 'others': y.append('others')
        else: y.append(row[-1])
        X.append(temp)
print(X[0])
X = np.array(X)
y = np.array(y)
print(X.shape)
print(y.shape)
print(X[0])
print(y[0])

[30.351930629205487, -102.85932046564082, -97.80066536077324, -5.058655104867581, 99.48688372906244, -10.117310209735162, -217.522169509306, 89.36957351932728, 124.78015925340034, 124.78015925340034, -193.91511235325729, 224.26704298246278, 109.6041939387976, 214.14973277272762, 107.24401847923033, 152.47927372823614, 136.94750970764866, 137.24823151613884, 90.6489414955104, 163.55962047831983, 218.26398878588063, 189.92624550665172, 100.61965527615514, 170.3624618870691, 0, 0, 1]
(11616, 27)
(11616,)
[  30.35193063 -102.85932047  -97.80066536   -5.0586551    99.48688373
  -10.11731021 -217.52216951   89.36957352  124.78015925  124.78015925
 -193.91511235  224.26704298  109.60419394  214.14973277  107.24401848
  152.47927373  136.94750971  137.24823152   90.6489415   163.55962048
  218.26398879  189.92624551  100.61965528  170.36246189    0.
    0.            1.        ]
others


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1)
first=svm.SVC(kernel=kernel,C=C,gamma=gamma)
first.fit(X_train,y_train)
first.predict(X_test)
print(first.score(X_train,y_train))
print(first.score(X_test, y_test))

0.9761813659843123
0.9767641996557659


In [5]:
# file_name = './訓練用影片/after_transform_1to15_head_and_shoulder.csv'
# X = []
# y = []

# #讀取訓練資料內容
# with open(file_name, newline='') as csvfile:
#     rows = csv.reader(csvfile)
#     for row in rows:
#         temp = []
#         for i in range(len(row)-3):
#             temp.append(float(row[i]))
#         #將資料'左中右'轉成onehotencoding
#         if row[-3] == 'left':
#             temp.append(1)
#             temp.append(0)
#             temp.append(0)
#         elif row[-3] == 'right':
#             temp.append(0)
#             temp.append(0)
#             temp.append(1)
#         elif row[-3] == 'middle':
#             temp.append(0)
#             temp.append(1)
#             temp.append(0)
#         temp.append(row[-2])
#         y.append(row[-1])
#         X.append(temp)
# X = np.array(X)
# y = np.array(y)
# print(X[0])
# print(y[0])

In [6]:
file_name = './訓練用影片/after_transform_1to15_head_and_shoulder.csv'
X = []
y = []

#讀取訓練資料內容
with open(file_name, newline='') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        if row[-1] != 'others':
            temp = []
            for i in range(len(row)-3):
                temp.append(float(row[i]))
            #將資料'左中右'轉成onehotencoding
            if row[-3] == 'left':
                temp.append(1)
                temp.append(0)
                temp.append(0)
            elif row[-3] == 'right':
                temp.append(0)
                temp.append(0)
                temp.append(1)
            elif row[-3] == 'middle':
                temp.append(0)
                temp.append(1)
                temp.append(0)
            temp.append(row[-2])
            y.append(row[-1])
            X.append(temp)
X = np.array(X)
y = np.array(y)
print(X[0])
print(y[0])

['-21.818181818181817' '-90.9090909090909' '-101.81818181818181'
 '-5.454545454545454' '100.0' '0.0' '-43.63636363636363'
 '-118.18181818181817' '9.09090909090909' '-118.18181818181817'
 '-65.45454545454545' '-107.27272727272727' '58.18181818181818'
 '-112.72727272727272' '103.4957332807958' '0.9090909090909091'
 '1.1818181818181819' '1.1818181818181819' '1.0727272727272728'
 '1.1272727272727272' '10.909090909090908' '5.454545454545454' '1' '0' '0'
 '1.8181818181818181']
looking


In [ ]:
print(kernel)
print(C)
print(gamma)
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=0)
print('1')
second=svm.SVC(kernel=kernel,C=C,gamma=gamma)
print('11')
second.fit(X_train,y_train)
print('12345')
second_result = second.predict(X_test)
print(second.score(X_train,y_train))
print(second.score(X_test, y_test))

linear
1
auto
1
11


In [ ]:
#計算兩點之間的距離
def get_distance(A, B):
    distance = math.pow((A[0] - B[0]), 2) + math.pow((A[1] - B[1]), 2)
    distance = math.sqrt(distance)
    return distance

In [ ]:
#計算兩個向量之間的夾角
def get_angle(first, second, angle):
    x1 = second[0] - first[0]
    y1 = second[1] - first[1]

    if angle == 0:
        x2 = 1
        y2 = 0
    elif angle == 90:
        x2 = 0
        y2 = 1
    elif angle == 180:
        x2 = -1
        y2 = 0
    elif angle == 270:
        x2 = 0
        y2 = -1
    dot = x1*x2+y1*y2
    det = x1*y2-y1*x2
    theta = np.arctan2(det, dot)
    theta = theta if theta>0 else 2*np.pi+theta
    theta = theta*180/np.pi
    if theta == 360: theta = 0.0
    
    return theta

In [ ]:
def get_angle_version_2(first, second, third, forth):
    x1 = second[0] - first[0]
    y1 = second[1] - first[1]
    x2 = forth[0] - third[0]
    y2 = forth[1] - third[1]
    dot = x1*x2+y1*y2
    det = x1*y2-y1*x2
    theta = np.arctan2(det, dot)
    theta = theta if theta>0 else 2*np.pi+theta
    theta = theta*180/np.pi
    if theta == 360: theta = 0.0

    return theta

In [ ]:
def infer_fast(net, img, net_input_height_size, stride, upsample_ratio,
               pad_value=(0, 0, 0), img_mean=np.array([128, 128, 128], np.float32), img_scale=np.float32(1/256)):
    height, width, _ = img.shape
    scale = net_input_height_size / height
    # scaled_img = img
    scaled_img = cv2.resize(img, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    scaled_img = normalize(scaled_img, img_mean, img_scale)
    min_dims = [net_input_height_size, max(scaled_img.shape[1], net_input_height_size)]
    padded_img, pad = pad_width(scaled_img, stride, pad_value, min_dims)

    tensor_img = torch.from_numpy(padded_img).permute(2, 0, 1).unsqueeze(0).float()
    tensor_img = tensor_img

    tensor_img = tensor_img.cuda()
    
    stages_output = net(tensor_img)

    stage2_heatmaps = stages_output[-2]
    heatmaps = np.transpose(stage2_heatmaps.squeeze().cpu().data.numpy(), (1, 2, 0))
    heatmaps = cv2.resize(heatmaps, (0, 0), fx=upsample_ratio, fy=upsample_ratio, interpolation=cv2.INTER_CUBIC)

    stage2_pafs = stages_output[-1]
    pafs = np.transpose(stage2_pafs.squeeze().cpu().data.numpy(), (1, 2, 0))
    pafs = cv2.resize(pafs, (0, 0), fx=upsample_ratio, fy=upsample_ratio, interpolation=cv2.INTER_CUBIC)
    
    return heatmaps, pafs, scale, pad

In [ ]:
#讀取影片和json檔
cap = cv2.VideoCapture(video_name)
net = PoseEstimationWithMobileNet()
checkpoint = torch.load('./checkpoint_iter_370000.pth', map_location='cpu')
load_state(net, checkpoint)

In [ ]:
def run_demo(net, image_provider, height_size):
    net = net.eval()
    net = net.cuda()
    cnt = 0
    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    delay = 1
    past_time = time.time()
    while 1:
        now_time = time.time()
#         print(now_time - past_time)
        past_time = now_time
        cnt += 1
        ret, img = image_provider.read()
        if not ret: break
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = infer_fast(net, img, height_size, stride, upsample_ratio)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
            pose = Pose(pose_keypoints, pose_entries[n][18])
            current_poses.append(pose)

        for pose in current_poses:
            pose.draw(img)
            
        img = cv2.addWeighted(orig_img, 0.6, img, 0.4, 0)
        for pose in current_poses:
            cv2.rectangle(img, (pose.bbox[0], pose.bbox[1]),
                          (pose.bbox[0] + pose.bbox[2], pose.bbox[1] + pose.bbox[3]), (0, 255, 0))
            
        for pose in current_poses:
            max_x = 0
            max_y = 0
            min_x = 9999
            min_y = 9999
            temp = []
            people = pose.keypoints
            for i in people:
                if i[0] != -1:
                    if i[0] > max_x: max_x = i[0]
                    if i[0] < min_x: min_x = i[0]
                    if i[1] > max_y: max_y = i[1]
                    if i[1] < min_y: min_y = i[1]
            if people[1][0] == -1 or people[5][0] == -1: continue
            if get_distance(people[1], people[5]) == 0: continue
            scale = NORMAL_LENGTH / get_distance(people[1], people[5])
            for i in KEYPOINTS:
                if i != 1:
                    if people[i][0] == -1:
                        temp.append(-1)
                        temp.append(-1)
                    else:
                        temp.append((people[i][0] - people[1][0]) * scale)
                        temp.append((people[i][1] - people[1][1]) * scale)
            for i in BONES:
                if people[i[0]][0] == -1 or people[i[1]][0] == -1:
                    temp.append(-1)
                else:
                    temp.append(get_distance(people[i[0]], people[i[1]]) * scale)
            for i in ANGLES:
                if people[i[0][0]][0] == -1 or people[i[0][1]][0] == -1:
                    temp.append(-1)
                else:
                    temp.append(get_angle(people[i[0][0]], people[i[0][1]], i[1]))
            if people[1][0] < (FRAME_WIDTH / 3):
                temp.append(1)
                temp.append(0)
                temp.append(0)
            elif people[1][0] > (FRAME_WIDTH / 3 * 2):
                temp.append(0)
                temp.append(0)
                temp.append(1)
            else:
                temp.append(0)
                temp.append(1)
                temp.append(0)
            temp = [temp]
            result = first.predict(temp)
            if result[0] =='asking' or result[0] == 'boring':
#                 print(result[0])
                cv2.putText(img, result[0], (int((min_x + max_x)/2), min_y-5), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 1, cv2.LINE_AA)
            else:
                temp = []
                for i in KEYPOINTS_4dots:
                    if people[i][0] == -1:
                        temp.append(0.0)
                        temp.append(0.0)
                    else:
                        temp.append((people[i][0] - people[1][0]) * scale)
                        temp.append((people[i][1] - people[1][1]) * scale)
                #身體角度
                for i in [[[1, 0], 0]]:
                    if people[i[0][0]][0] == -1 or people[i[0][1]][0] == -1:
                        temp.append(0.0)
                    else:
                        temp.append(get_angle(people[i[0][0]], people[i[0][1]], i[1]))
                
                #頭和肩膀的比例
                if people[0][1] != -1: temp.append((people[1][1] - people[0][1]) / get_distance(people[1], people[5]))
                else: temp.append(0.0)
                
                #眼睛和耳朵的高度(y)和肩膀的比例
                if people[14][1] != -1: temp.append((people[1][1] - people[14][1]) / get_distance(people[1], people[5]))
                else: temp.append(0.0)
                if people[15][1] != -1: temp.append((people[1][1] - people[15][1]) / get_distance(people[1], people[5]))
                else: temp.append(0.0)
                if people[16][1] != -1: temp.append((people[1][1] - people[16][1]) / get_distance(people[1], people[5]))
                else: temp.append(0.0)
                if people[17][1] != -1: temp.append((people[1][1] - people[17][1]) / get_distance(people[1], people[5]))
                else: temp.append(0.0)
                
                #眼睛和耳朵的高度差(調過大小的)
                if people[14][1] != -1 and people[16][1] != -1: temp.append((people[16][1] - people[14][1]) * scale)
                else: temp.append(0.0)
                if people[15][1] != -1 and people[17][1]!= -1: temp.append((people[17][1] - people[15][1]) * scale)
                else: temp.append(0.0)
                if people[1][0] < (FRAME_WIDTH / 3):
                    temp.append(1)
                    temp.append(0)
                    temp.append(0)
                elif people[1][0] > (FRAME_WIDTH / 3 * 2):
                    temp.append(0)
                    temp.append(0)
                    temp.append(1)
                else:
                    temp.append(0)
                    temp.append(1)
                    temp.append(0)
                temp.append(scale)
                
                temp = [temp]
                result = second.predict(temp)
                cv2.putText(img, result[0], (int((min_x + max_x)/2), min_y-5), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 255), 1, cv2.LINE_AA)
                
        #img = cv2.resize(img, (1280, 720), interpolation=cv2.INTER_AREA)
        
        if cnt % CUT_PHOTO == 0: cv2.imwrite('./results/' + json_name + '/svm_new/' + str(int(cnt/CUT_PHOTO)) + '.jpg', img)


In [ ]:
run_demo(net, cap, 256)

In [ ]:
# def run_demo(net, image_provider, height_size):
#     net = net.eval()
      net = net.cuda()
#     cnt = 0
#     stride = 8
#     upsample_ratio = 4
#     num_keypoints = Pose.num_kpts
#     previous_poses = []
#     delay = 1
#     while 1:
#         cnt += 1
#         ret, img = image_provider.read()
#         if not ret: break
#         orig_img = img.copy()
#         heatmaps, pafs, scale, pad = infer_fast(net, img, height_size, stride, upsample_ratio)

#         total_keypoints_num = 0
#         all_keypoints_by_type = []
#         for kpt_idx in range(num_keypoints):  # 19th for bg
#             total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

#         pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs)
#         for kpt_id in range(all_keypoints.shape[0]):
#             all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
#             all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
#         current_poses = []
#         for n in range(len(pose_entries)):
#             if len(pose_entries[n]) == 0:
#                 continue
#             pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
#             for kpt_id in range(num_keypoints):
#                 if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
#                     pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
#                     pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
#             pose = Pose(pose_keypoints, pose_entries[n][18])
#             current_poses.append(pose)

#         for pose in current_poses:
#             pose.draw(img)
        
#         img = cv2.addWeighted(orig_img, 0.6, img, 0.4, 0)
#         for pose in current_poses:
#             cv2.rectangle(img, (pose.bbox[0], pose.bbox[1]),
#                           (pose.bbox[0] + pose.bbox[2], pose.bbox[1] + pose.bbox[3]), (0, 255, 0))
#         for pose in current_poses:
#             #這裡辨識
        
#         if cnt % CUT_PHOTO == 0: cv2.imwrite('./results/' + json_name + '/CNN/CNN_' + str(int(cnt/CUT_PHOTO)) + '.jpg', img)